# Week8 Homework
## Working with Strings

The data frame contains notes taken by staff members. Your task is to clean it up to make it usable for analysis. 

### Tasks
* Create start date, end date columns and populate the values (If notes don't include month or day use 1 for missing value) 

i.e: 
    
    - 2018 => 2018-01-01  
    - June 2021 => 2021-06-01  
    - 2020 => 2020-01-01

* Create phone number column (If notes don't include phone numbers, use None)
* Create a column `duration` showing difference between start & end dates in years as a whole number

Note: Choose one of the following datasets to work on based on your taste! 

In [56]:
import pandas as pd

In [57]:
spicy = 'https://raw.githubusercontent.com/msaricaumbc/DS_Data/master/strings/hw_data_spicy.csv'
less_spicy = 'https://raw.githubusercontent.com/msaricaumbc/DS_Data/master/strings/hw_data_less_spicy.csv'

In [58]:
df = pd.read_csv(spicy, index_col=0)

In [59]:
df.head()

,email,notes
0,ayalajodi@hotmail.com,start date 1982-05-20 end dt 2016
1,tmitchell@hotmail.com,P# 896.729.1768x061-STDT 2004-06-12-end date 2...
2,johnrivera@gmail.com,st date:1976/end dt 05/16/2019/pn: (895)351-43...
3,kimberly60@yahoo.com,stdt1995-03 end date December 2021 pn: 881139...
4,paulharding@jefferson-burns.com,pn: (448)006-0055x240-STDT 1983-02-ed date:08/...


In [70]:
import re
start =[];
end=[];
phone_no = [];
for i in df['notes']:
  ends = re.sub('end dt|end date|ed date|endt|end',"end date", i) #End date d1
  starts = re.sub('start date|STDT|st date|stdt|start', 'start date', ends)#Start date d2
  D3 = re.sub('P#|pn|p#|p/n', 'PHONE', starts)#Phone number
  idED=D3.find('end date')
  idSD=D3.find('start date')
  idP=D3.find('PHONE')
  if('PHONE' in D3):
    if(D3[0] =='P' ):
      phone_no.append(D3[13:idSD-1].strip())
    else:
      phone_no.append(D3[idP+13:].strip())
  else:
    phone_no.append('None')
  D4 = D3[idSD:idED-1]
  D5 = D3[idED:]
  if('PHONE' in D5):
    D5 = D5
  else:
    D5 = (D5+' '+'PHONE')
  idx3 = D5.find('PHONE')
  end_date = D5[9:idx3-1]
  start_date = D4[D4.find('start date') + 10:].replace(":","").strip()
  try:
    start.append(pd.to_datetime(start_date))
  except Exception:
    start.append(pd.to_datetime(start_date[:len(start_date)-2] + str(int(start_date[len(start_date)-2:]) +2000)))
  try:
    end.append(pd.to_datetime(end_date))
  except Exception:
    end.append(pd.to_datetime(end_date[:len(end_date)-2] + str(int(end_date[len(end_date)-2:]) +2000)))

In [71]:
A = pd.DataFrame({'START_DATE': start, 'END_DATE': end})
A

,START_DATE,END_DATE
0,1982-05-20,2016-01-01
1,2004-06-12,2012-05-27
2,1976-01-01,2019-05-16
3,1995-03-01,2021-12-01
4,1983-02-01,2018-08-29
...,...,...
1995,2005-11-01,2013-08-01
1996,2008-09-01,2016-03-01
1997,1987-05-01,2018-07-10
1998,2004-03-01,2016-12-12


In [72]:
p = re.compile(r'((e:)|t)(\s*)?(\w*)?(\D*)?(\d*)?(-|\/)?(\d*)?(-|\/)?(\d*)?',re.I)
df['phonenumber'] = [re.sub(p,'',x) for x in df['notes']]
phone = re.compile(r'\(?|\d*\)?\-?x?',re.I)
df['phonenumber'] = [phone.findall(p) for p in df['phonenumber']]
df['phonenumber'] = [(''.join(x)) for x in df['phonenumber']]
df['phonenumber']=df['phonenumber'].replace(r'^\s*$|^-$', 'None', regex=True)
p4=re.compile(r'^-')
df['phonenumber']=[re.sub(p4,'',x) for x in df['phonenumber']]
p5 = re.compile(r'(\b^[0-9]{2}(\/)[0-9]{2})$\b')
df

,email,notes,phonenumber
0,ayalajodi@hotmail.com,start date 1982-05-20 end dt 2016,None
1,tmitchell@hotmail.com,P# 896.729.1768x061-STDT 2004-06-12-end date 2...,8967291768x061--
2,johnrivera@gmail.com,st date:1976/end dt 05/16/2019/pn: (895)351-43...,(895)351-4305x228
3,kimberly60@yahoo.com,stdt1995-03 end date December 2021 pn: 881139...,8811396378
4,paulharding@jefferson-burns.com,pn: (448)006-0055x240-STDT 1983-02-ed date:08/...,(448)006-0055x240-
...,...,...,...
1995,shane14@hotmail.com,STDT 11/05 end dt 08/13,None
1996,whebert@cohen.com,STDT 2008-09/ed date:03/16,None
1997,timothyjohnson@rice.org,st date:05/87/end date 2018-07-10,None
1998,sellersjeffrey@williams.info,st date:March 2004/end dt 12/12/2016,None


In [65]:
df3 = pd.DataFrame({"Duration": (A.END_DATE-A.START_DATE).astype('timedelta64[Y]')})
df3.astype(int)

,Duration
0,33
1,7
2,43
3,26
4,35
...,...
1995,7
1996,7
1997,31
1998,12
